In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
from Preprocessing import merge_dfs

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from itertools import combinations

In [ ]:

class FeatureSelector():
    def __init__(self, model, X_train, y_train, max_combos, features, splits=5) -> None:
        self.model = model
        self.X = X_train
        self.y = y_train
        self.splits = splits
        self.max_combos = max_combos
        self.features = features # list containing names of the features you want to select from
        self.features_to_drop = None # name of the optimal combination of features
        self.best_score = 10000

    def test_performance(self, X_variant):
        """Performs K-fold to test performance of a dataframe"""
        scores = cross_val_score(self.model, X_variant, self.y, cv=self.splits, scoring='neg_mean_squared_error')
        return np.sqrt(-np.mean(scores))


    def find_combos(self):
        """
        0. create a list 
        1. loops trough n_combos
        2. append to list all combos of size of loop+1 (0,1,2)
        3. return list
        """
        combo_list = []
        for i in range(self.max_combos):
            combinations_list = list(combinations(self.features, i))
            for combination in combinations_list:
                combo_list.append(list(combination))
        return combo_list


    def fit(self) -> None:
        """
        1. runds find combos
        2. loops trough all elements in find combos list 
        3. create a copy of X
        4. remove the columns from the copy
        5. test performance on the new df
        6. if lower than best_SCORE save
        """
        combo_list = self.find_combos()
        for combination in combo_list:
            X_copy = self.X.copy()
            X_copy.drop(columns=combination, inplace=True)
            score = self.test_performance(X_copy)
            if score < self.best_score:
                self.best_score = score
                self.features_to_drop = combination
    
    def get_best(self):
        new_X = self.X.drop(columns=self.features_to_drop)
        return self.features_to_drop, new_X


In [ ]:
from Preprocessing import merge_dfs    
from sklearn.model_selection import train_test_split
df = merge_dfs()

# Lagre 2023 data til senere
data_2023 = df[df['Aarstall'] == 2023].drop(columns=['Trafikkmengde'])

# Droppe kolonner der trafikkmengde er nan
data = df[df['Aarstall'] != 2023]
data = data.dropna(subset=['Trafikkmengde'])

# Dele i features og target, og splitte datasettet i trenings-og testdata
X = data.drop(columns=['Trafikkmengde'])
y = data['Trafikkmengde']

# Splitter dataen
X_train, X_test, y_train, y_test = train_test_split(X, y ,shuffle=False, test_size=0.3)


In [ ]:
X_train.columns

In [ ]:
features = ['Solskinstid', 'Lufttemperatur', 'Vindstyrke', 'Lufttrykk', 'Vindkast',
       'Globalstraling', 'Vindretning']

all_features = X_train.columns

In [ ]:
import pickle
model = pickle.load(open('model.pkl', 'rb'))




In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer()

In [ ]:
type(X_train)

In [ ]:
X_train = imputer.fit_transform(X_train)

In [ ]:
type(X_train)

In [ ]:
X_train = pd.DataFrame(X_train, columns=all_features)

In [ ]:
feature_selector = FeatureSelector(model, X_train, y_train, max_combos=3, features=features)
feature_selector.fit()